In [1]:
import json
import csv
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from os.path import exists
import statistics

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
comp=pd.read_json("drive/MyDrive/MSC_Onlab/open-data-master/data/competitions.json")

comp.dtypes

competition_id                int64
season_id                     int64
country_name                 object
competition_name             object
competition_gender           object
competition_youth              bool
competition_international      bool
season_name                  object
match_updated                object
match_updated_360            object
match_available_360          object
match_available              object
dtype: object

In [4]:
comp =  comp[(comp.competition_gender == "male")]
comp =  comp[comp["season_name"].str.contains("2022") | comp["season_name"].str.contains("2020") | comp["season_name"].str.contains("2020") | comp["season_name"].str.contains("2019") | comp["season_name"].str.contains("2018")]

In [16]:
comp[["competition_name","season_name","season_id"]]

,competition_name,season_name,season_id
1,Champions League,2018/2019,4
2,Champions League,2017/2018,1
26,FIFA World Cup,2022,106
27,FIFA World Cup,2018,3
34,Indian Super league,2021/2022,108
35,La Liga,2020/2021,90
36,La Liga,2019/2020,42
37,La Liga,2018/2019,4
38,La Liga,2017/2018,1
62,UEFA Euro,2020,43


In [19]:
comp.describe

<bound method NDFrame.describe of     competition_id  season_id   country_name     competition_name  \
1               16          4         Europe     Champions League   
2               16          1         Europe     Champions League   
26              43        106  International       FIFA World Cup   
27              43          3  International       FIFA World Cup   
34            1238        108          India  Indian Super league   
35              11         90          Spain              La Liga   
36              11         42          Spain              La Liga   
37              11          4          Spain              La Liga   
38              11          1          Spain              La Liga   
62              55         43         Europe            UEFA Euro   

   competition_gender  competition_youth  competition_international  \
1                male              False                      False   
2                male              False                      Fa

In [5]:
#Egy olyan lista létrehozása, aminek elemei szintén listák és minden ilyen allista egy versenysorozatot tartalmaz. Az allisták egy eleme egy meccs.
competitionList = []
for index, row in comp.iterrows():
  matchesDF=pd.read_json("drive/MyDrive/MSC_Onlab/open-data-master/data/matches/" + str(row["competition_id"]) + "/" + str(row["season_id"]) + ".json")
  matchDFList = []
  for index, row2 in matchesDF.iterrows():
    matchDF = pd.read_json("drive/MyDrive/MSC_Onlab/open-data-master/data/events/" + str(row2["match_id"]) +".json")
    matchID = [row2["match_id"]] * len(matchDF)
    competitionID = [row["competition_id"]] * len(matchDF)
    competitionName = [row["competition_name"]] * len(matchDF)
    matchDF["match_id"] = matchID
    matchDF["competition_id"] = competitionID
    matchDF["competition_name"] = competitionName
    matchDFList.append(matchDF)
  competitionList.append(matchDFList)

In [10]:
competition_DF_list = []
for compet in competitionList:
  competDF = pd.concat(compet)
  competition_DF_list.append(competDF)


In [33]:
competition_DF_list[0].dtypes


id                          object
index                        int64
period                       int64
timestamp           datetime64[ns]
minute                       int64
second                       int64
type                        object
possession                   int64
possession_team             object
play_pattern                object
team                        object
duration                   float64
tactics                     object
related_events              object
player                      object
position                    object
location                    object
pass                        object
carry                       object
ball_receipt                object
under_pressure             float64
duel                        object
foul_committed              object
shot                        object
goalkeeper                  object
off_camera                 float64
clearance                   object
foul_won                    object
interception        

In [12]:
competition_DF_list[0]["type"].value_counts()

{'id': 30, 'name': 'Pass'}                 890
{'id': 42, 'name': 'Ball Receipt*'}        846
{'id': 43, 'name': 'Carry'}                669
{'id': 17, 'name': 'Pressure'}             314
{'id': 2, 'name': 'Ball Recovery'}         100
{'id': 4, 'name': 'Duel'}                   56
{'id': 9, 'name': 'Clearance'}              51
{'id': 23, 'name': 'Goal Keeper'}           38
{'id': 16, 'name': 'Shot'}                  30
{'id': 6, 'name': 'Block'}                  25
{'id': 38, 'name': 'Miscontrol'}            22
{'id': 3, 'name': 'Dispossessed'}           19
{'id': 10, 'name': 'Interception'}          15
{'id': 22, 'name': 'Foul Committed'}        15
{'id': 14, 'name': 'Dribble'}               15
{'id': 21, 'name': 'Foul Won'}              12
{'id': 39, 'name': 'Dribbled Past'}         10
{'id': 33, 'name': '50/50'}                  6
{'id': 19, 'name': 'Substitution'}           6
{'id': 28, 'name': 'Shield'}                 5
{'id': 18, 'name': 'Half Start'}             4
{'id': 40, 'n

In [14]:
sum(competition_DF_list[0]["type"].isna())

0